# Train a customized model

In [1]:
import os
import tensorflow as tf
from tensorflow.python.keras import layers
from tensorflow import keras
import glob
%load_ext autoreload
%autoreload 2
%matplotlib inline

Maybe you need to define a new model strucure, we can inherit class `BaseSRModel`.

Here we use `SRCNN` as example:

## Model establishing

In [2]:
from src.model import BaseSRModel

**It's noted that:**

- In original paper, the author use "valid" padding when training, yet "same" padding when testing. Here as an example, we set padding "valid" and thus the label size should be modified later. See http://mmlab.ie.cuhk.edu.hk/projects/SRCNN.html for details

- The `SRCNN` model pre-defined in module `models` use "same" padding by default.

- Learning rate of the last layer in origional paper is set to 1e-5, others are set to 1e-4. I don't know how to do different learning rate for different layers in tensorflow... So I set all to 1e-5 for convenience...

In [3]:
class SRCNN(BaseSRModel):
    """
    Using 9-1-5 model.
    """

    def __init__(self, scale, model_name, channel=1):

        super(SRCNN, self).__init__(scale, model_name, channel)
        
        # kernel size and number
        self.f1 = 9
        self.f2 = 1
        self.f3 = 5

        self.n1 = 64
        self.n2 = 32

    def create_model(self, load_weights=False, weights_path=None):

        inp = super(SRCNN, self).create_model()

        x = layers.Convolution2D(self.n1, (self.f1, self.f1),
                                 activation='relu', padding='valid', name='level1')(inp)
        x = layers.Convolution2D(self.n2, (self.f2, self.f2),
                                 activation='relu', padding='valid', name='level2')(x)

        out = layers.Convolution2D(self.channel, (self.f3, self.f3),
                                   padding='valid', name='output')(x)

        model = keras.Model(inp, out)

        if load_weights:
            weights_path = self.weights_path if weights_path is None else weights_path
            model.load_weights(self.weight_path)
            print("loaded model %s from %s" % (self.model_name, weights_path))

        self.model = model
        return self

    def lr_schedule(self, epoch, *args, **kwargs):
        return 1e-5

## Data Pipeline

**Pre-work is same as `train_models` notebook.**

In [4]:
train_dir = "./Image/set14" # Arbitrary
valid_dir = "./Image/set5"
AUTOTUNE = tf.data.experimental.AUTOTUNE
SCALE = 3

In [5]:
from src.write2tfrec import write_dst_tfrec, load_tfrecord

cache_dir = "./cache"
os.makedirs(cache_dir, exist_ok=True)

In [6]:
if not os.path.isfile("./cache/set14_train_48x48.tfrec"):
    paths = list(glob.glob(os.path.join(train_dir, "*")))
    write_dst_tfrec(paths, 10, 48, "./cache/set14_train_48x48.tfrec")
    
if not os.path.isfile("./cache/set5_valid_48x48.tfrec"):
    paths = list(glob.glob(os.path.join(valid_dir, "*")))
    write_dst_tfrec(paths, 10, 48, "./cache/set5_valid_48x48.tfrec")


**Preprocessing function here is a little different:**
    
- Lr-image should be upsample with `bicubic` kernel
- Considering the padding issue, we need to center-crop Hr-patch to the same size as output of the model.

In [7]:
from src.preprocess import degrade_image
from src.data_utils import center_crop

def preprocess(hr):
    lr, hr = degrade_image(hr, SCALE, method=2, restore_shape=True)
    hr = center_crop(hr, (36, 36))
    return lr, hr

In [9]:
trdst = load_tfrecord(48, "./cache/set14_train_48x48.tfrec").map(preprocess).repeat()
valdst = load_tfrecord(48, "./cache/set5_valid_48x48.tfrec").map(preprocess)

## Training

In [10]:
model = SRCNN(scale=SCALE, model_name="SRCNN",
                      channel=3).create_model(load_weights=False,
                                              weights_path=None)
model.fit(trdst,
          valdst,
          nb_epochs=2,
          steps_per_epoch=20,
          batch_size=16,
          use_wn=False)

Training model : SRCNN_X3


W0621 21:30:27.754841  2524 training_utils.py:1353] Expected a shuffled dataset but input dataset `x` is not shuffled. Please invoke `shuffle()` on input dataset.


Epoch 1/2
 1/20 [>.............................] - ETA: 50s - loss: 0.2249 - psnr_tf: 7.6447

W0621 21:30:30.805746  2524 callbacks.py:236] Method (on_train_batch_end) is slow compared to the batch update (0.143106). Check your callbacks.


17/20 [========================>.....] - ETA: 24s - loss: 0.2299 - psnr_tf: 7.58 - ETA: 8s - loss: 0.2364 - psnr_tf: 8.1106 - ETA: 3s - loss: 0.2472 - psnr_tf: 7.623 - ETA: 1s - loss: 0.2378 - psnr_tf: 7.938 - ETA: 0s - loss: 0.2419 - psnr_tf: 7.7502
Epoch 00001: saving model to ./weights/SRCNN_X3.h5
20/20 [==============================] - 3s 161ms/step - loss: 0.2369 - psnr_tf: 7.9468 - val_loss: 0.1941 - val_psnr_tf: 8.9553
Epoch 2/2
17/20 [========================>.....] - ETA: 0s - loss: 0.1449 - psnr_tf: 8.850 - ETA: 0s - loss: 0.2276 - psnr_tf: 7.572 - ETA: 0s - loss: 0.2074 - psnr_tf: 8.410 - ETA: 0s - loss: 0.2153 - psnr_tf: 8.129 - ETA: 0s - loss: 0.2038 - psnr_tf: 8.554 - ETA: 0s - loss: 0.2090 - psnr_tf: 8.3432
Epoch 00002: saving model to ./weights/SRCNN_X3.h5
20/20 [==============================] - 1s 26ms/step - loss: 0.2030 - psnr_tf: 8.3938 - val_loss: 0.1595 - val_psnr_tf: 9.7366
